In [2]:
from ultralytics import YOLO
import pandas as pd
import os

# --- CẤU HÌNH ---
MODELS_DIR = '/workspace/advance_cv_project/train/runs/detect'

# Danh sách 6 model của bạn
model_names = [
    'sroie_yolov8n_finetune', 'sroie_yolov8m_finetune', 'sroie_yolov9c_finetune',
    'receipt_yolov8n_finetune', 'receipt_yolov8m_finetune', 'receipt_yolov9c_finetune'
]

# Đường dẫn data test
DATASETS = {
    'SROIE_Test': '/workspace/advance_cv_project/Datasets/SROIE_YOLO_TRAIN/data.yaml',
    'Custom_Receipt_Test': '/workspace/advance_cv_project/Datasets/RECEIPT_YOLO_TRAIN/data.yaml' 
}

results = []

print("--- BẮT ĐẦU BENCHMARK TOÀN DIỆN (ACCURACY + SPEED + SIZE) ---")

for model_folder in model_names:
    weights_path = os.path.join(MODELS_DIR, model_folder, 'weights', 'best.pt')
    
    if not os.path.exists(weights_path):
        print(f"Skipping {model_folder} (Not found)")
        continue
        
    print(f"\nEvaluating: {model_folder}...")
    try:
        # 1. Load Model
        model = YOLO(weights_path)
        
        # 2. Lấy thông tin kích thước File (Size MB)
        size_mb = os.path.getsize(weights_path) / (1024 * 1024)
        
        # 3. Lấy số lượng tham số (Params)
        # model.info() trả về (layers, params, gradients, flops)
        n_params = sum(x.numel() for x in model.parameters()) / 1e6 # Triệu tham số
        
        # 4. Chạy đánh giá trên từng tập dữ liệu
        for dataset_name, yaml_path in DATASETS.items():
            print(f"  -> Testing on {dataset_name}...")
            
            # Chạy Val/Test
            metrics = model.val(
                data=yaml_path,
                split='test',
                imgsz=1024,
                batch=8,
                verbose=False,
                plots=False
            )
            
            # 5. Lấy Tốc độ (Inference Time)
            # metrics.speed trả về dictionary {'preprocess': ..., 'inference': ..., 'postprocess': ...}
            inference_time = metrics.speed['inference']
            
            results.append({
                'Model': model_folder,
                'Test_Data': dataset_name,
                
                # --- NHÓM CHẤT LƯỢNG ---
                'mAP50-95 (%)': round(metrics.box.map * 100, 2),
                'mAP50 (%)': round(metrics.box.map50 * 100, 2),
                'Recall (%)': round(metrics.box.mr * 100, 2),
                'Precision (%)': round(metrics.box.mp * 100, 2),
                
                # --- NHÓM HIỆU NĂNG ---
                'Speed (ms)': round(inference_time, 2),
                'Size (MB)': round(size_mb, 2),
                'Params (M)': round(n_params, 2)
            })
            
    except Exception as e:
        print(f"  -> Lỗi: {e}")

# Xuất kết quả
df = pd.DataFrame(results)

# Sắp xếp theo Data Test rồi đến mAP50-95
df = df.sort_values(by=['Test_Data', 'mAP50-95 (%)'], ascending=[True, False])

print("\n=== BẢNG TỔNG HỢP ===")
print(df.to_string())

# df.to_csv('final_full_benchmark.csv', index=False)
# print("\nĐã lưu kết quả vào 'final_full_benchmark.csv'")

--- BẮT ĐẦU BENCHMARK TOÀN DIỆN (ACCURACY + SPEED + SIZE) ---

Evaluating: sroie_yolov8n_finetune...
  -> Testing on SROIE_Test...
Ultralytics 8.3.234 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA A40, 45403MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.4±3.0 ms, read: 78.3±57.2 MB/s, size: 397.3 KB)
val: Scanning /workspace/advance_cv_project/Datasets/SROIE_YOLO_TRAIN/test/labels.cache... 347 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 347/347 17.1Mit/s 0.0s
val: /workspace/advance_cv_project/Datasets/SROIE_YOLO_TRAIN/test/images/X51005230659.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 44/44 15.0it/s 2.9s
                   all        347      18704      0.962      0.958      0.978      0.708
Speed: 0.7ms preprocess, 1.7ms inference, 0.0ms loss, 0.7ms postprocess per image
  -> Testing on Custom_Receipt_Test.